---
# 신경망 종합 테스트
- burst 하여 active cell 이 많으면 새로운 패턴을 인지한다는 의미
- 먼저 spatial pooler 가 안정되어야 한다.
---
encoder - sp - tm -> prediction + textual representation<br/>
예측 cell / 활성화 cell 비교 - 예측이 맞는지 확인 가능<br/>
예측 column / 입력 column 비교 - 마찬가지<br/>
burst 한 column / 활성화 된 column - 마찬가지<br/>


In [1]:
import numpy as np
import random
import nbimporter
import TM
import Encoder
import SP3

Importing Jupyter notebook from TM.ipynb
Importing Jupyter notebook from memoryStruct.ipynb
Importing Jupyter notebook from Encoder.ipynb
Importing Jupyter notebook from SP3.ipynb


In [2]:
encoder = Encoder.ScalarEncoder(out_size=100, w=20, max_val=30)
sp = SP3.SpatialPooler(encoder.out_size, columns=512, perm_conn=.6, minOver=encoder.w/5, potential_rate=0.9)

In [3]:
for epoch in range(5):
    a = np.arange(encoder.max_val)
    random.shuffle(a)
    
    for i in a:
        input_data = encoder.encode(i)
        sp.compute_SDR(input_data)

In [6]:
for i in range(encoder.max_val):
    sp.compute_SDR(encoder.encode(i), learn=False)
    sdr1 = sp.getActiveColumns()
    sp.compute_SDR(encoder.encode(i+1), learn=False)
    sdr2 = sp.getActiveColumns()
    
    sp.similarity(sdr1, sdr2, False)
    print('')

유사도 : 0.7
[154 172 206 212 217 253 282 374 464 492]
[154 206 212 253 282 314 332 374 386 388 464]

유사도 : 0.36363636363636365
[154 206 212 253 282 314 332 374 386 388 464]
[ 90 111 183 199 212 214 253 290 314 317 386 430]

유사도 : 0.25
[ 90 111 183 199 212 214 253 290 314 317 386 430]
[ 90 201 290 297 325 382 399 430 449 482]

유사도 : 0.4
[ 90 201 290 297 325 382 399 430 449 482]
[154 156 164 201 209 297 382 398 465 482]

유사도 : 0.0
[154 156 164 201 209 297 382 398 465 482]
[104 195 224 296 318 367 389 401 475 486]

유사도 : 0.4
[104 195 224 296 318 367 389 401 475 486]
[ 14 192 224 302 318 355 359 389 401 455]

유사도 : 0.5
[ 14 192 224 302 318 355 359 389 401 455]
[ 73 156 219 224 339 355 389 401 455 486]

유사도 : 0.1
[ 73 156 219 224 339 355 389 401 455 486]
[ 17  73  83 117 127 176 191 192 255 378]

유사도 : 0.2
[ 17  73  83 117 127 176 191 192 255 378]
[ 57  83  94 148 255 358 380 413 433 497]

유사도 : 0.3
[ 57  83  94 148 255 358 380 413 433 497]
[ 17  83 133 240 255 292 366 413 442 503 507]

유사도 :

In [80]:
tm = TM.TemporalMemory(sdr_size=sp.columnCount, max_seg=5, threshold=7)

In [115]:
tm.learn = True

seq_data = [1,8,5,4,2,3,9,5,3]

for i in range(3):
    for s in seq_data:
        dat = encoder.encode(s)
        sp.compute_SDR(dat, learn=False)
        sdr = sp.getActiveColumns_raw()
        tm.compute(sdr)

segment 225 destroyed
segment 302 destroyed
segment 305 destroyed
segment 102 destroyed
segment 308 destroyed
segment 310 destroyed
segment 311 destroyed
segment 312 destroyed
segment 313 destroyed
segment 315 destroyed
segment 317 destroyed
segment 318 destroyed
segment 320 destroyed
segment 321 destroyed
segment 325 destroyed
segment 326 destroyed
segment 269 destroyed
segment 327 destroyed
segment 330 destroyed
segment 332 destroyed
segment 334 destroyed
segment 335 destroyed
segment 336 destroyed
segment 338 destroyed
segment 339 destroyed
segment 340 destroyed
segment 347 destroyed
segment 350 destroyed
segment 351 destroyed
segment 352 destroyed
segment 354 destroyed
segment 108 destroyed
segment 182 destroyed
segment 357 destroyed
segment 360 destroyed
segment 361 destroyed
segment 362 destroyed
segment 363 destroyed


In [116]:
tm.learn = False

test_data = [1,2,3,4,5,6,7,8,9]
sub = test_data[:5]

for d in seq_data:
    dat = encoder.encode(d)
    sp.compute_SDR(dat, learn=False)
    sdr = sp.getActiveColumns_raw()
    tm.compute(sdr)

    print("현재 sdr : {}".format(sp.getActiveColumns()))
    print("다음 예측 : {}".format(tm.column_predicted()))
    print("현재 burst : {}".format(tm.column_bursted()))
    print("\n")

현재 sdr : [ 40 128 175 272 279 292 348 392 402 444]
다음 예측 : [258, 386, 132, 392, 266, 139, 14, 143, 402, 23, 407, 416, 55, 185, 443, 190, 191, 197, 454, 72, 459, 333, 334, 91, 220, 349, 479, 362, 237, 241, 117, 374]
현재 burst : [128, 292, 175, 272, 279, 444]


현재 sdr : [ 91 139 143 190 191 197 220 258 266 386 443 454 459]
다음 예측 : [128, 386, 132, 392, 14, 402, 23, 416, 292, 425, 308, 55, 185, 197, 72, 334, 348, 349, 362, 241, 117]
현재 burst : []


현재 sdr : [ 14  23  55  72 117 132 185 197 241 349 386]
다음 예측 : [416, 128, 407, 292, 37, 40, 392, 479, 362, 333, 334, 175, 272, 402, 211, 279, 444, 348, 349, 445]
현재 burst : [241]


현재 sdr : [ 37 211 333 334 349 362 407 416 445 479]
다음 예측 : [416, 128, 386, 425, 266, 362, 334, 47, 432, 241, 20, 180, 214, 348, 212, 308, 316]
현재 burst : [416, 362]


현재 sdr : [128 292 308 334 348 362 392 402 416 425]
다음 예측 : [386, 132, 14, 402, 407, 416, 37, 54, 445, 197, 454, 455, 77, 334, 333, 205, 211, 212, 91, 349, 479, 362, 237, 241, 116, 374]
현재 burst : [292, 39

In [15]:
# len(tm.connInfo.totalSegments)
cell = 2000
print('{} 번 cell 에 연결되어 있는 synapse : {}'.format(cell, tm.connInfo.totalCells[cell].connectedSynapses))
#print(len(tm.prev_activeCells))
#print(len(tm.activeCells))
#for c in tm.connInfo.totalCells:
 #   if(len(c.connectedSynapses) > 0):
  #      print("살아있는 cell : " + str(c.cell_id))
        
#choose = list(tm.connInfo.totalCells[cell].connectedSynapses)[0]
#for i in tm.connInfo.totalSegments:
 #   for j in i.synapses:
  #      if (j.synapse_id == choose):
   #         print(j.synapse_id)
    #        print(j.preSynapticCell)
     #       print(j.permanence)
        
print(sp.getActiveColumns())
print(tm.predicted)
print(tm.bursted)

2000 번 cell 에 연결되어 있는 synapse : set()
[ 86 160 176 237 353 361 415 458 492 500]
[131, 285, 329, 361, 176, 415, 86, 361, 458, 492, 237, 353, 285, 462, 40, 285, 361, 40, 145, 451, 462, 500, 40]
[86, 237, 353, 361, 458, 492]
